# Task 2: Interactive Dashboard for Retail Sales Insights

This dashboard visualizes key findings from the EDA of the Online Retail Dataset, including top-selling products, sales by country, sales trends over time, and customer spending patterns. It uses Dash and Plotly for interactivity, with dropdowns to filter by country.

In [6]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Load dataset
df = pd.read_csv("Cleaned_OnlineRetail.csv")

# Ensure correct data types
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Sales'] = df['Quantity'] * df['UnitPrice']

# Initialize app
app = Dash(__name__)

# App Layout
app.layout = html.Div([
    html.H1("📊 Online Retail Dashboard", 
            style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': 30}),
    
    # Country Filter
    html.Div([
        html.Label("🌍 Select Country:", style={'fontWeight': 'bold'}),
        dcc.Dropdown(
            id="country-filter",
            options=[{"label": c, "value": c} for c in sorted(df['Country'].unique())],
            value="United Kingdom",
            multi=False,
            style={"width": "50%"}
        )
    ], style={"display": "flex", "justifyContent": "center", "marginBottom": "30px"}),

    # First row: Two graphs side by side
    html.Div([
        html.Div([dcc.Graph(id="sales-histogram")], 
                 style={"flex": "50%", "padding": "10px"}),
        html.Div([dcc.Graph(id="category-bar")], 
                 style={"flex": "50%", "padding": "10px"}),
    ], style={"display": "flex", "flexWrap": "wrap"}),

    # Second row: Full width time-series
    html.Div([
        dcc.Graph(id="time-series")
    ], style={"padding": "10px"})
])

# Callbacks
@app.callback(
    [Output("sales-histogram", "figure"),
     Output("category-bar", "figure"),
     Output("time-series", "figure")],
    [Input("country-filter", "value")]
)
def update_dashboard(selected_country):
    dff = df[df["Country"] == selected_country]

    # Histogram: Customer Sales Distribution
    fig1 = px.histogram(dff, x="Sales", nbins=50, 
                        title="Distribution of Customer Sales",
                        color_discrete_sequence=["#3498db"])
    fig1.update_layout(margin=dict(l=20, r=20, t=50, b=20))

    # Bar Chart: Top 10 Products by Sales
    top_products = dff.groupby("Description")["Sales"].sum().nlargest(10).reset_index()
    fig2 = px.bar(top_products, x="Description", y="Sales", 
                  title="Top 10 Products by Sales",
                  color="Sales", color_continuous_scale="Blues")
    fig2.update_layout(xaxis_tickangle=-45, margin=dict(l=20, r=20, t=50, b=120))

    # Line Chart: Sales Over Time
    monthly_sales = dff.resample("ME", on="InvoiceDate")["Sales"].sum().reset_index()
    fig3 = px.line(monthly_sales, x="InvoiceDate", y="Sales", 
                   title="Monthly Sales Trend",
                   markers=True, color_discrete_sequence=["#e74c3c"])
    fig3.update_layout(margin=dict(l=20, r=20, t=50, b=20))

    return fig1, fig2, fig3


if __name__ == "__main__":
    app.run(debug=True)


# 📊 Dashboard Overview

The dashboard includes:

- **Top 10 Products by Sales**: Bar chart, filterable by country via dropdown.  
- **Customer Spending Distribution**: Histogram of sales amounts per transaction, filterable by country.  
- **Sales Over Time**: Line chart of monthly sales trends, filterable by country.  

---

## 🚀 How to Use
1. Run the Python script or the code cell containing the task2.  
2. Open [http://127.0.0.1:8050/](http://127.0.0.1:8050/) in your browser.  
3. Use the **dropdown menu** to select a country.  
   - The bar chart, histogram, and line chart will update accordingly.  
4. To stop the server:  
   - Press **CTRL+C** in the terminal (if running script), or  
   - **Interrupt the kernel** in Jupyter Notebook.  

---

## 🎥 Screencast
A demo video (`dashboard_demo.mp4`) showcases the dashboard in action, including country filtering and interactive features.

 [Launch demo notebook in Binder] https://mybinder.org/v2/gh/Hamna902/NEXUS-AI-ML-Tasks/b70aec0c146abdf59956d4b6b00452763d48ec07?urlpath=lab%2Ftree%2Ftask2-checkpoint.ipynb